<a href="https://colab.research.google.com/github/dp-08/Gen-AI/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Configuration & Hyperparameters ---
IMG_SIZE = 64         # Image resolution (DCGAN is best for 64x64 or 128x128)
CHANNELS = 3          # RGB images
LATENT_DIM = 100      # Size of the random noise vector
BATCH_SIZE = 128
EPOCHS = 100          # Training for thousands of epochs is common for good results
ADAM_LR = 0.0002
ADAM_BETA1 = 0.5      # Recommended for GANs

# --- 1. Data Loading and Preprocessing (Using a simplified CelebA path) ---
def load_and_preprocess_data(data_dir, img_size, batch_size):
    """Loads image data, resizes, and normalizes it to [-1, 1]."""
    print(f"Loading images from directory: {data_dir}")

    # The CelebA dataset must be manually downloaded and extracted.
    # Assumes 'data_dir' contains the image files.
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        label_mode=None,           # Unlabeled data for GANs
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=True,
        interpolation='bilinear'
    )

    # Normalize images from [0, 255] to [-1, 1]
    dataset = dataset.map(lambda x: (x - 127.5) / 127.5)
    return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Replace this with the actual path to your extracted CelebA images
# DATASET_PATH = 'path/to/img_align_celeba'
# For demonstration, assume a simplified placeholder
DATASET_PATH = 'celeba_dataset_placeholder'

# NOTE: You must replace 'celeba_dataset_placeholder' with the actual path
# to your CelebA image directory for the code to run.
if not os.path.isdir(DATASET_PATH):
    print("Warning: DCGAN requires a downloaded and extracted dataset (e.g., CelebA) at the specified path.")
    print("Skipping data loading for conceptual code structure.")
    # Create a dummy dataset for structure demonstration if path is missing
    dummy_data = np.random.rand(100, IMG_SIZE, IMG_SIZE, CHANNELS)
    dummy_data = (dummy_data - 0.5) * 2.0
    dataset = tf.data.Dataset.from_tensor_slices(dummy_data).batch(BATCH_SIZE)
else:
    dataset = load_and_preprocess_data(DATASET_PATH, IMG_SIZE, BATCH_SIZE)


# --- 2. Generator Model ---
def make_generator_model(latent_dim, channels):
    """
    Creates the Generator network using Conv2DTranspose (deconvolutional layers).
    Maps from latent vector (1D) to image (3D).
    """
    model = tf.keras.Sequential(name="generator")

    # Start with a Dense layer, reshape to a small image volume (4x4x512)
    # The input to the first Conv2DTranspose will be 4x4.
    model.add(layers.Dense(4 * 4 * 512, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((4, 4, 512))) # (4, 4, 512)

    # Upsample to 8x8
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU()) # (8, 8, 256)

    # Upsample to 16x16
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU()) # (16, 16, 128)

    # Upsample to 32x32
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU()) # (32, 32, 64)

    # Upsample to 64x64 (final layer)
    model.add(layers.Conv2DTranspose(channels, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    # (64, 64, 3) with pixel values in [-1, 1]

    return model

# --- 3. Discriminator Model ---
def make_discriminator_model(img_size, channels):
    """
    Creates the Discriminator network using Conv2D layers.
    Maps from image (3D) to a single probability (0=fake, 1=real).
    """
    model = tf.keras.Sequential(name="discriminator")

    # Downsample from 64x64 to 32x32
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                            input_shape=[img_size, img_size, channels]))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3)) # (32, 32, 64)

    # Downsample to 16x16
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3)) # (16, 16, 128)

    # Downsample to 8x8
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3)) # (8, 8, 256)

    # Downsample to 4x4
    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3)) # (4, 4, 512)

    # Flatten and output probability
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model


# --- 4. Define Loss and Optimizers ---
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    # Real images should be labeled 1 (real_output near 1)
    real_loss = cross_entropy(tf.ones_like(real_output) * 0.9, real_output) # Use label smoothing (0.9 instead of 1.0)
    # Fake images should be labeled 0 (fake_output near 0)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # Generator wants discriminator to think fakes are real (label 1)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(ADAM_LR, beta_1=ADAM_BETA1)
discriminator_optimizer = tf.keras.optimizers.Adam(ADAM_LR, beta_1=ADAM_BETA1)


# --- 5. DCGAN Model (Combined with a custom train step) ---
# Use a custom Model class to implement the GAN training logic within the Keras fit() loop

class DCGAN(tf.keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(DCGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(DCGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn
        self.d_loss_metric = tf.keras.metrics.Mean(name='d_loss')
        self.g_loss_metric = tf.keras.metrics.Mean(name='g_loss')

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]

        # 1. Train the Discriminator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generate fake images
            generated_images = self.generator(random_latent_vectors, training=True)

            # Get the discriminator's predictions
            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            # Calculate losses
            d_loss = self.d_loss_fn(real_output, fake_output)
            g_loss = self.g_loss_fn(fake_output)

        # Apply gradients to Discriminator
        d_gradients = disc_tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_gradients, self.discriminator.trainable_variables))

        # Apply gradients to Generator
        g_gradients = gen_tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_gradients, self.generator.trainable_variables))

        # Update and return the metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}


# --- 6. Initialization and Training ---

# Build the models
generator = make_generator_model(LATENT_DIM, CHANNELS)
discriminator = make_discriminator_model(IMG_SIZE, CHANNELS)

# Instantiate the custom DCGAN model
dcgan = DCGAN(discriminator=discriminator, generator=generator, latent_dim=LATENT_DIM)

# Compile the DCGAN
dcgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    d_loss_fn=discriminator_loss,
    g_loss_fn=generator_loss
)

print("Starting DCGAN training...")

# NOTE: This will only work if the dataset is correctly loaded.
# Use a GPU/Colab for efficient training.
# history = dcgan.fit(dataset, epochs=EPOCHS)

print("DCGAN model structure defined and compiled. Ready for training using dcgan.fit(dataset, epochs=EPOCHS).")
print(f"Generator output shape: {generator.output_shape}")
print(f"Discriminator output shape: {discriminator.output_shape}")

# Example generation after hypothetical training
# sample_noise = tf.random.normal(shape=[1, LATENT_DIM])
# generated_image = generator(sample_noise, training=False)
# plt.imshow((generated_image[0] * 0.5 + 0.5))
# plt.show()

Skipping data loading for conceptual code structure.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Starting DCGAN training...
DCGAN model structure defined and compiled. Ready for training using dcgan.fit(dataset, epochs=EPOCHS).
Generator output shape: (None, 64, 64, 3)
Discriminator output shape: (None, 1)
